<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/09%20-%20GAN/02_GAN2(2_1ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

GAN 모델을 이용하여 원하는 손글씨 숫자를 생성하는 모델을 만들기

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

TensorFlow 2.x selected.


## 데이터 불러오기

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (_, _) = mnist.load_data()

x_train = x_train.reshape(60000, 28*28).astype('float32') / 255
y_train = tf.one_hot(y_train, depth=10).numpy()

## 모델 구성

### 옵션 설정

In [ ]:
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

epochs = 100
batch_size = 100
buffer_size = 60000
learning_rate = 0.0002

### 신경망 구성

In [ ]:
# 처음에 만든 신경망과 크게 다르지 않다. 라벨 정보가 들어갈 

# 생성기(G) 모델 신경망
def generator():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(n_hidden, activation='relu', input_shape=(n_noise+n_class,)),
    tf.keras.layers.Dense(n_input, activation='sigmoid')
  ])

  return model

# 판별기(G) 모델 신경망
def discriminator():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(n_hidden, activation='relu', input_shape=(n_input+n_class,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  return model

G = generator()
D = discriminator()

G.summary()
D.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               35584     
_________________________________________________________________
dense_1 (Dense)              (None, 784)               201488    
Total params: 237,072
Trainable params: 237,072
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 256)               203520    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 203,777
Trainable params: 203,777
Non-trainable params: 0
_________________________________________________________________


### 손실 함수

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# 생성기(G) 손실 함수
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

# 판별기(D) 손실 함수
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  return real_loss + fake_loss

### 최적화 설정

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate)

### 모델 학습 함수

In [ ]:
# 모델 학습 함수
@tf.function
def train_step(images, labels):
  # 임의의 
  noise = tf.random.normal([batch_size, n_noise])
  noise = tf.concat([noise, labels], axis=1)

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = G(noise, training=True)

    real_output = D(tf.concat([images, labels], axis=1), training=True)
    fake_output = D(tf.concat([generated_images, labels], axis=1), training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gen_gradient = gen_tape.gradient(gen_loss, G.trainable_variables)
  disc_gradient = disc_tape.gradient(disc_loss, D.trainable_variables)

  generator_optimizer.apply_gradients(zip(gen_gradient, G.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(disc_gradient, D.trainable_variables))

  return (gen_loss, disc_loss)

### 이미지 출력 함수

In [ ]:
#이미지 출력 함수
def show_images(test_image):
  input_size = len(test_image)

  predictions = G(test_image, training=False)

  fig = plt.figure(figsize=(input_size, 1))
  for i in range(input_size):
    plt.subplot(1, input_size, i+1)
    plt.imshow(tf.reshape(predictions[i], (28, 28)))
    plt.axis('off')
    plt.title(str(tf.math.argmax(test_image.numpy()[i][128:]).numpy()))

### 모델 학습 및 이미지 확인

In [ ]:
num_examples_to_generate = 16

# 데이터를 섞고 배치크기로 나눠준다.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size).batch(batch_size)
train_dataset = list(train_dataset.as_numpy_iterator())

y_test = y_train[:num_examples_to_generate]
test_noise = tf.random.normal([num_examples_to_generate, n_noise])
test_noise = tf.concat([test_noise, y_test], 1)

for epoch in range(epochs):
  print('Epoch:', '%04d' % (epoch+1))
  for data in train_dataset:
    gen_loss, disc_loss = train_step(data[0], data[1])

  if epoch % 10 == 0:
    show_images(test_noise)

  print('D loss: {:.4}'.format(disc_loss),
        'G loss: {:.4}'.format(gen_loss))

show_images(test_noise)

In [ ]:
noise = np.random.uniform(-1., 1., size=[10, 10])
labels = [[0,0,0,0,0,0,0,0,0,1] for _ in range(10)]
inputs = tf.concat([noise, labels], 1)
print(inputs[0])
asd

tf.Tensor(
[-0.06426246 -0.34010086 -0.11410678  0.73011312 -0.1391853  -0.74418949
  0.00174777 -0.19019187  0.84862746 -0.1546649   0.          0.
  0.          0.          0.          0.          0.          0.
  0.          1.        ], shape=(20,), dtype=float64)


NameError: ignored

## 전체 코드

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

mnist = tf.keras.datasets.mnist

(x_train, y_train), (_, _) = mnist.load_data()

x_train = x_train.reshape(60000, 28*28).astype('float32') / 255
y_train = tf.one_hot(y_train, depth=10).numpy()

#########
# 옵션 설정
######

n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

epochs = 200
batch_size = 100
buffer_size = 60000
learning_rate = 0.0002

#########
# 신경망 모델 구성
######

# 생성기(G) 모델 신경망
def generator():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(n_hidden, activation='relu', input_shape=(n_noise+n_class,)),
    tf.keras.layers.Dense(n_input, activation='sigmoid')
  ])

  return model

# 판별기(G) 모델 신경망
def discriminator():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(n_hidden, activation='relu', input_shape=(n_input+n_class,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  return model

G = generator()
D = discriminator()

# G.summary()
# D.summary()

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# 생성기(G) 손실 함수
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

# 판별기(D) 손실 함수
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  return real_loss + fake_loss

# 최적화 설정
generator_optimizer = tf.keras.optimizers.Adam(learning_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate)

#########
# 신경망 모델 학습
######

# 모델 학습 함수
@tf.function
def train_step(images, labels):
  noise = tf.random.normal([batch_size, n_noise])
  noise = tf.concat([noise, labels], axis=1)

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = G(noise, training=True)

    real_output = D(tf.concat([images, labels], axis=1), training=True)
    fake_output = D(tf.concat([generated_images, labels], axis=1), training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gen_gradient = gen_tape.gradient(gen_loss, G.trainable_variables)
  disc_gradient = disc_tape.gradient(disc_loss, D.trainable_variables)

  generator_optimizer.apply_gradients(zip(gen_gradient, G.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(disc_gradient, D.trainable_variables))

  return (gen_loss, disc_loss)

#이미지 출력 함수
def show_images(test_image):
  input_size = len(test_image)

  predictions = G(test_image, training=False)

  fig = plt.figure(figsize=(input_size, 1))
  for i in range(input_size):
    plt.subplot(1, input_size, i+1)
    plt.imshow(tf.reshape(predictions[i], (28, 28)))
    plt.axis('off')
    plt.title(str(tf.math.argmax(test_image.numpy()[i][128:]).numpy()))

# 모델 학습 및 이미지 확인
num_examples_to_generate = 16

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size).batch(batch_size)
train_dataset = list(train_dataset.as_numpy_iterator())

y_test = y_train[:num_examples_to_generate]
test_noise = tf.random.normal([num_examples_to_generate, n_noise])
test_noise = tf.concat([test_noise, y_test], 1)

for epoch in range(epochs):
  print('Epoch:', '%04d' % (epoch+1))
  for data in train_dataset:
    gen_loss, disc_loss = train_step(data[0], data[1])

  if epoch % 10 == 0:
    show_images(test_noise)

  print('D loss: {:.4}'.format(disc_loss),
        'G loss: {:.4}'.format(gen_loss))

show_images(test_noise)
